In [1]:
# module 20 final project part one - import CSVs and do some simple ETL to prepare for upload to PostGres
# working locally in mlenv environment means needed packages are already installed 

# import json - we might need this later if we do web scraping
import pandas as pd
import numpy as np



In [2]:
# take the file path from the windows explorer and change the back slash to forward slashes

file_path = "./Resources/Basic_Stats.csv"
NFL_basic_stats_df = pd.read_csv(file_path, low_memory=False)

# since the pd.read.csv can not find the file path from Google Collab, I will try it from local VM server instead to see if it behaves differently
# getting a permission error - has this ever happened before?  Do I have to open the file in Excel and enable editing?

In [3]:
# ETL section
# file has over 17000 records
# do we need to keep retired players? 
# do we care about age column since it has nulls? drop the whole column? 
NFL_basic_stats_df.head(10)
# we could probably drop birthday too but actually we probably won't use any of this data to compute models
# this file is being used really only as a test to download, clean, and load into postGres3
# definitely have to drop any columns with nulls in them, not the rows


Age        Birth Place    Birthday           College Current Status  \
0   NaN  Grand Rapids , MI   5/23/1921        Notre Dame        Retired   
1   NaN        Dayton , OH  12/21/1930            Dayton        Retired   
2  56.0        Temple , TX   9/11/1960    Louisiana Tech        Retired   
3  30.0   New Orleans , LA   9/30/1986               LSU        Retired   
4  25.0       Detroit , MI   3/31/1992  Central Michigan         Active   
5   NaN        Sumner , IL   9/11/1892          Illinois        Retired   
6   NaN        Casper , WY   9/16/1920         Wisconsin        Retired   
7  22.0       Navarre , FL   1/31/1995           Clemson         Active   
8   NaN                NaN         NaN        No College        Retired   
9  98.0      Alverton , PA   2/23/1919          Duquesne        Retired   

          Current Team  Experience  Height (inches)     High School  \
0                  NaN   3 Seasons             71.0             NaN   
1                  NaN    1 Season             70.0             NaN   
2                  NaN    1 Season             74.0             NaN   
3                  NaN   5 Seasons             73.0             NaN   
4  Pittsburgh Steelers  3rd season             77.0  Clintondale HS   
5                  NaN    1 Season             73.0             NaN   
6                  NaN   3 Seasons             73.0             NaN   
7        New York Jets      Rookie             77.0      Navarre HS   
8                  NaN    1 Season              NaN             NaN   
9                  NaN    1 Season             70.0             NaN   

          High School Location             Name  Number  \
0                          NaN      Evans, Fred     NaN   
1                          NaN       Raiff, Jim     NaN   
2                          NaN    Fowler, Bobby     NaN   
3                          NaN   Johnson, Quinn     NaN   
4  Clinton Twp.,Macomb Co., MI     Walton, L.T.    96.0   
5                          NaN      Petty, Ross     NaN   
6                          NaN      Farris, Tom     NaN   
7                           FL  Leggett, Jordan     NaN   
8                          NaN  Thompson, Pinky     NaN   
9                          NaN    Gonda, George     NaN   

               Player Id Position  Weight (lbs) Years Played  
0      fredevans/2513736      NaN         185.0  1946 - 1948  
1       jimraiff/2523700      NaN         235.0  1954 - 1954  
2    bobbyfowler/2514295      NaN         230.0  1985 - 1985  
3     quinnjohnson/79593      NaN         255.0  2009 - 2013  
4     l.t.walton/2552444       DE         305.0          NaN  
5      rosspetty/2523136      NaN         180.0  1920 - 1920  
6      tomfarris/2513861      NaN         185.0  1946 - 1948  
7  jordanleggett/2557885       TE         250.0          NaN  
8  pinkythompson/2527303      NaN           NaN  1927 - 1927  
9    georgegonda/2515030      NaN         175.0  1942 - 1942

In [4]:
# some info about this file
NFL_basic_stats_df.info()
# name player_id and Current Status are non null
# player Id is a unique key that can be used as primary key if we move to PostGres

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17172 entries, 0 to 17171
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Age                   13504 non-null  float64
 1   Birth Place           14794 non-null  object 
 2   Birthday              16835 non-null  object 
 3   College               17133 non-null  object 
 4   Current Status        17172 non-null  object 
 5   Current Team          3096 non-null   object 
 6   Experience            17133 non-null  object 
 7   Height (inches)       17026 non-null  float64
 8   High School           2514 non-null   object 
 9   High School Location  2510 non-null   object 
 10  Name                  17172 non-null  object 
 11  Number                1708 non-null   float64
 12  Player Id             17172 non-null  object 
 13  Position              3096 non-null   object 
 14  Weight (lbs)          17121 non-null  float64
 15  Years Played       

In [31]:
# ETL Monday
# to populate the table in PostGres lets just get enough test records to work with
# IDEA just use active players (as of 4 years ago when this dataset was created)

# df.sort_values(by=['col1'])



In [16]:
NFL_df2 = NFL_basic_stats_df.sort_values(by=["Current Status"])

In [68]:
NFL_df3 = NFL_df2[100:285]

In [69]:
NFL_df4 = NFL_df3.drop(columns=["Years Played","High School","High School Location","Age","Birth Place"])

In [82]:
NFL_df4 = NFL_df4[NFL_df4["Number"].notnull()]
# data = data[data['sms'].notnull()]
NFL_df4 = NFL_df4[NFL_df4["Birthday"].notnull()]
NFL_df4.sort_values(by=["Number"])
# confirmed Number is Jersey Number- it should be integer not Float64 but its not a show stopper to fix it now
# Example Sam Martin Detroit Lions Punter wears Jersey # 6
# when importing to pgAdmin the height. number and weight all came in as integers because of how I defined the table columns


Birthday                   College CurStatus               CurTeam  \
15016    1/6/1994             Florida State    Active  Tampa Bay Buccaneers   
2228    3/20/1985                 Tennessee    Active      Cleveland Browns   
15039   2/27/1990         Appalachian State    Active         Detroit Lions   
4286     2/8/1990              Oregon State    Active      Los Angeles Rams   
2678   11/15/1992  Southeast Missouri State    Active      Los Angeles Rams   
...           ...                       ...       ...                   ...   
11964   7/10/1989                California    Active    New Orleans Saints   
6377    2/18/1989             Florida State    Active   San Francisco 49ers   
7842    9/21/1991  Northwest Missouri State    Active      Los Angeles Rams   
2655   12/22/1987                       USC    Active     Minnesota Vikings   
15365   8/16/1993                  Arkansas    Active  New England Patriots   

       Experience  Height               Name  Number              Player_Id  \
15016  3rd season    76.0    Winston, Jameis     3.0  jameiswinston/2552033   
2228   8th season    74.0  Colquitt, Britton     4.0  brittoncolquitt/71259   
15039  5th season    73.0        Martin, Sam     6.0      sammartin/2541311   
4286   6th season    77.0     Hekker, Johnny     6.0   johnnyhekker/2535663   
2678   1st season    74.0    McRoberts, Paul     8.0  paulmcroberts/2556615   
...           ...     ...                ...     ...                    ...   
11964  7th season    76.0    Jordan, Cameron    94.0  cameronjordan/2495184   
6377   5th season    76.0    Carradine, Tank    95.0  tankcarradine/2539224   
7842   3rd season    75.0     Longacre, Matt    96.0   mattlongacre/2553722   
2655   8th season    75.0   Griffen, Everson    97.0  eversongriffen/496790   
15365  3rd season    74.0      Flowers, Trey    98.0    treyflowers/2552278   

      Position  Weight  
15016       QB   231.0  
2228         P   205.0  
15039        P   211.0  
4286         P   236.0  
2678        WR   200.0  
...        ...     ...  
11964       DE   287.0  
6377        LB   295.0  
7842        DE   260.0  
2655        DE   273.0  
15365       DE   265.0  

[106 rows x 11 columns]

In [83]:
# df.rename(columns={"A": "a", "B": "c"})
# NFL_df4.rename(columns={"Current Status": "CurStatus", "Current Team": "CurTeam", "Height (inches)": "Height"})
NFL_df4 = NFL_df4.rename(columns={"Weight (lbs)": "Weight"})

# NFL_df4 = NFL_df4.rename(columns={"Player Id": "Player_Id", "Current Team": "CurTeam", "Height (inches)": "Height"})
# fix the column names to no blanks or parens
NFL_df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 10670 to 14996
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Birthday    106 non-null    object 
 1   College     106 non-null    object 
 2   CurStatus   106 non-null    object 
 3   CurTeam     106 non-null    object 
 4   Experience  106 non-null    object 
 5   Height      106 non-null    float64
 6   Name        106 non-null    object 
 7   Number      106 non-null    float64
 8   Player_Id   106 non-null    object 
 9   Position    106 non-null    object 
 10  Weight      106 non-null    float64
dtypes: float64(3), object(8)
memory usage: 9.9+ KB


In [81]:
# Next Steps - export to CSV
# example = pd.to_csv('C:/Path to save edited file.csv', index_col = False)
file_path_out = "./Resources/NFL_players_sample_cleaned.csv"
NFL_df4.to_csv(file_path_out, index = False)

In [84]:
# Next Steps - import into NFL_stats database, NFL_players table in postgres- done outside notebook
# confirmed the format of the output file
NFL_df4.head()

Birthday           College CurStatus               CurTeam  \
10670  4/29/1993        Notre Dame    Active        Houston Texans   
10668   8/5/1993        Washington    Active       Atlanta Falcons   
15366  10/3/1986  New Mexico State    Active   San Francisco 49ers   
15365  8/16/1993          Arkansas    Active  New England Patriots   
11964  7/10/1989        California    Active    New Orleans Saints   

       Experience  Height             Name  Number              Player_Id  \
10670  2nd season    76.0     Martin, Nick    66.0     nickmartin/2555189   
10668  2nd season    75.0    Perkins, Josh    82.0    joshperkins/2556444   
15366  6th season    74.0     Nelson, Kyle    86.0     kylenelson/2530722   
15365  3rd season    74.0    Flowers, Trey    98.0    treyflowers/2552278   
11964  7th season    76.0  Jordan, Cameron    94.0  cameronjordan/2495184   

      Position  Weight  
10670        C   299.0  
10668       TE   227.0  
15366       LS   240.0  
15365       DE   265.0  
11964       DE   287.0